In [1]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime
from lxml import html 
import requests
from bs4 import BeautifulSoup
import itertools
import requests
import json
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

In [2]:
import json

url = 'https://dat-tr-prda-ops-vava.azureedge.net/cars/56169/car-details-en.json'

response = requests.get(url).text
response_info = json.loads(response)
response2 = requests.get("https://dat-tr-prda-ops-vava.azureedge.net/cars/61426/car-details-en.json").text
response_info2 = json.loads(response2)


In [7]:
response3 = requests.get("https://dat-tr-prda-ops-vava.azureedge.net/cars/61126/car-details-en.json").text
response_info3 = json.loads(response3)
print(response_info3['carSpecification']['registrationDate'])

2018-11-15T00:00:00


## Creando Dataframe, sobre detalles generales del coche

In [128]:
marca=[]
marca.append(str(response_info2['carSpecification']['makeName']))
modelo=[]
modelo.append(str(response_info2['carSpecification']['modelName']))
detalles=[]
detalles.append(response_info2['carSpecification']['trimLevelName'])
Fecha=[]
Fecha.append(response_info2['carSpecification']['registrationDate'])

dtcPrice=[]
dtcPrice.append(str(response_info2['carSpecification']['dtcPrice']))
actualMileage=[]
actualMileage.append(str(response_info2['carSpecification']['actualMileage']))

combustible=[]
combustible.append(str(response_info2['carSpecification']['fuelTypeName']))
almacen=[]
almacen.append(str(response_info2['carSpecification']['warehouseName']))
ID=[]
ID.append(str(response_info2['carSpecification']['auctionId']))
imgcar=[]
imgcar.append(response_info2['mainImageUrl'])
df3 = pd.DataFrame(
    {'marca': marca,
     'modelo': modelo,
     'detalles': detalles,
     'Fecha': Fecha,
     'actualMileage':actualMileage,
     'Price':dtcPrice,
     'combustible': combustible,
     'almacen': almacen,
     'ID': ID,
     'imgcar':imgcar,
    })
df3['date'] = df3['Fecha'].astype('datetime64[ns]')
df3

,marca,modelo,detalles,Fecha,actualMileage,Price,combustible,almacen,ID,imgcar,date
0,Suzuki,Vitara,1.4 Boosterjet 4x2 Gl Plus Tek Renk 140HP,2022-02-02T00:00:00,41539,388000.0,Benzin,Basaksehir Warehouse,43922,https://prvtrprdaopsvava.blob.core.windows.net...,2022-02-02


## Construyendo dataframe condition, estará contenido en una columna del df_general

In [129]:
from PIL import Image, ImageDraw
import unicodedata
import glob,os
import re
lista=[]
listai=[]
#pensar en incluir tb los elementos subelemtos que tienen 0 inspecciones para ello habria que meter todas las listas en un if inspeccion==0
cochecondition_elemento=[]
cochecondition_subelemento=[]
cochecondition_inspecciones=[]
cochecondition_orden_inspecciones=[]
cochecondition_informe=[]
cochecondition_detalle=[]
cochecondition_imagen=[]
cochecondition_fileimagen=[]
for i,pos in enumerate(response_info2['conditionSummary'].values()):


            for i2,pos2 in enumerate(pos[0]['inspectionCheckPointsGroups']): #son 38 diccionarios
                
                #print("Estamos en el elemento "+str(i2)+" llamado "+ pos2['checkpointGroupName'] + " q tiene "+str(len(pos2['inspectionCheckPointsGroups']))+" subelementos")#cada diccionario tiene una lista de diccionarios, componentes del elemento en cuestion
                
                #hay que iterar cada uno de los subelementos y ver si se han chequeado
                for i3,pos3 in enumerate(pos2['inspectionCheckPointsGroups']):
                    
                #lo siguiente es mirar si hay detalles de inspecciones, y lo siguiente es mirar tb info acerca de inspecciones 
                    inspecciones=0
                    #ver inspecciones de todos print("El subelemento "+ pos3["checkpointGroupName"]+" tiene "+str(len(pos3["inspectionCheckPoints"]))+"inspecciones")
                    for i4,pos4 in enumerate(pos3['inspectionCheckPoints']):
                        
                        
                        inspecciones=inspecciones+1

                        cochecondition_elemento.append({'Elemento': pos2['checkpointGroupName']})
                        cochecondition_subelemento.append({'Subelemento': pos3["checkpointGroupName"]})
                        cochecondition_inspecciones.append({'nºInspecciones(totales)': len(pos3["inspectionCheckPoints"])})
                        cochecondition_informe.append({'Informe de inspección':pos4['checkpointName']})
                        cochecondition_orden_inspecciones.append({'Orden Inspección': inspecciones})
                        #print("hay "+str(len(pos4['inspectionCheckPointDetails']))+" detalles") habra x detalles pero solo nos quedamos con el true
                        for i5,pos5 in enumerate(pos4['inspectionCheckPointDetails']):
                            if pos5['isChecked']==True:
                                if pos5['checkpointDetailName']=='Yes':
                                    #print(pos5['checkpointDetailName']+", "+pos4['checkpointName']+" inspección chequeada")
                                    
                                    cochecondition_detalle.append({'Detalle inspección':pos4["checkpointName"]+"; "+pos5['checkpointDetailName']})
                                    subpil=""
                                else:
                                    
                                    cochecondition_detalle.append({'Detalle inspección':pos5['checkpointDetailName']})
                                    subpil=pos5['checkpointDetailName']
                                    
                                cochecondition_imagen.append({'nºImagenes':len(pos4['documents'])})
                                
                                if len(pos4['documents']) >=1:

                                    cochecondition_fileimagen.append({'file_imagen':pos4['documents'][0]['documentPath']})
                                else:
                                    cochecondition_fileimagen.append({'file_imagen':'No_hay_Imagen'})
elementos=pd.DataFrame(cochecondition_elemento)
subelementos=pd.DataFrame(cochecondition_subelemento)
inspecciones=pd.DataFrame(cochecondition_inspecciones)
Orden_inspeccion=pd.DataFrame(cochecondition_orden_inspecciones)
informe=pd.DataFrame(cochecondition_informe)
detalle=pd.DataFrame(cochecondition_detalle)
imagen=pd.DataFrame(cochecondition_imagen)
fileimagen=pd.DataFrame(cochecondition_fileimagen)
#pd.concat([elementos,subelementos,inspecciones])
#pd.concat([elementos,subelementos,inspecciones], axis=0, ignore_index=True)
df1=pd.concat([elementos,subelementos,inspecciones,Orden_inspeccion,informe,detalle,imagen,fileimagen], axis=1)
df1

,Elemento,Subelemento,nºInspecciones(totales),Orden Inspección,Informe de inspección,Detalle inspección,nºImagenes,file_imagen
0,Front Bumper,Grill,1,1,Damage,Damage; Yes,1,https://prvtrprdaopsvava.blob.core.windows.net...
1,Front Bumper,Front Bumper,2,1,Scratch,greater than 6 inches / 150mm,1,https://prvtrprdaopsvava.blob.core.windows.net...
2,Front Bumper,Front Bumper,2,2,Chipped,1-5,0,No_hay_Imagen
3,R front wing,R Front Fender,1,1,Scratch,less than 1.5 inches/37.5mm,1,https://prvtrprdaopsvava.blob.core.windows.net...
4,R Front rim,R Front rim,1,1,Scratch,less than 6 inches/150mm,1,https://prvtrprdaopsvava.blob.core.windows.net...
5,Roof,Roof,1,1,Bird Drop,Bird Drop; Yes,1,https://prvtrprdaopsvava.blob.core.windows.net...
6,Front Windscreen,Front Windscreen,1,1,Stone chip on glass,1-5,1,https://prvtrprdaopsvava.blob.core.windows.net...
7,R Front door,R Front door,1,1,Dent with scratch,less than 1.5 inches/37.5mm,1,https://prvtrprdaopsvava.blob.core.windows.net...
8,R rear rim,R rear rim,1,1,Scratch,less than 6 inches/150mm,1,https://prvtrprdaopsvava.blob.core.windows.net...
9,Rear bumper,Rear bumper,1,1,Damage,Damage ; Yes,1,https://prvtrprdaopsvava.blob.core.windows.net...


In [130]:
df3['condittion_summary']=[df1] 
df3


#administrativo y contencioso(implica juicio)administrativo el contecioso es irse a tribunales, el otro es seguir en via administrativa

,marca,modelo,detalles,Fecha,actualMileage,Price,combustible,almacen,ID,imgcar,date,condittion_summary
0,Suzuki,Vitara,1.4 Boosterjet 4x2 Gl Plus Tek Renk 140HP,2022-02-02T00:00:00,41539,388000.0,Benzin,Basaksehir Warehouse,43922,https://prvtrprdaopsvava.blob.core.windows.net...,2022-02-02,Elemento Subelemento nºInsp...


In [131]:
pd.concat([elementos.reset_index(drop=True),subelementos.reset_index(drop=True),inspecciones], axis=1)

,Elemento,Subelemento,nºInspecciones(totales)
0,Front Bumper,Grill,1
1,Front Bumper,Front Bumper,2
2,Front Bumper,Front Bumper,2
3,R front wing,R Front Fender,1
4,R Front rim,R Front rim,1
5,Roof,Roof,1
6,Front Windscreen,Front Windscreen,1
7,R Front door,R Front door,1
8,R rear rim,R rear rim,1
9,Rear bumper,Rear bumper,1


In [132]:
df3['condittion_summary'][0]

,Elemento,Subelemento,nºInspecciones(totales),Orden Inspección,Informe de inspección,Detalle inspección,nºImagenes,file_imagen
0,Front Bumper,Grill,1,1,Damage,Damage; Yes,1,https://prvtrprdaopsvava.blob.core.windows.net...
1,Front Bumper,Front Bumper,2,1,Scratch,greater than 6 inches / 150mm,1,https://prvtrprdaopsvava.blob.core.windows.net...
2,Front Bumper,Front Bumper,2,2,Chipped,1-5,0,No_hay_Imagen
3,R front wing,R Front Fender,1,1,Scratch,less than 1.5 inches/37.5mm,1,https://prvtrprdaopsvava.blob.core.windows.net...
4,R Front rim,R Front rim,1,1,Scratch,less than 6 inches/150mm,1,https://prvtrprdaopsvava.blob.core.windows.net...
5,Roof,Roof,1,1,Bird Drop,Bird Drop; Yes,1,https://prvtrprdaopsvava.blob.core.windows.net...
6,Front Windscreen,Front Windscreen,1,1,Stone chip on glass,1-5,1,https://prvtrprdaopsvava.blob.core.windows.net...
7,R Front door,R Front door,1,1,Dent with scratch,less than 1.5 inches/37.5mm,1,https://prvtrprdaopsvava.blob.core.windows.net...
8,R rear rim,R rear rim,1,1,Scratch,less than 6 inches/150mm,1,https://prvtrprdaopsvava.blob.core.windows.net...
9,Rear bumper,Rear bumper,1,1,Damage,Damage ; Yes,1,https://prvtrprdaopsvava.blob.core.windows.net...


## Construyendo dataframe mechanical, estará contenido en una columna del df_general

In [133]:
from PIL import Image, ImageDraw
import unicodedata
import glob,os
import re
lista=[]
listai=[]
cochemechanical_elemento=[]
cochemechanical_inspecciones=[]
cochemechanical_orden_inspecciones=[]
cochemechanical_informe=[]
cochemechanical_detalle=[]
cochemechanical_imagen=[]
cochemechanical_fileimagen=[]
for i,pos in enumerate(response_info2['mechanicalSummary'].values()):

            for i2,pos2 in enumerate(pos[0]['inspectionCheckPointsGroups']): #son 38 diccionarios

                inspecciones=0
                for i3,pos3 in enumerate(pos2['inspectionCheckPoints']):
                    inspecciones=inspecciones+1
                    cochemechanical_elemento.append({'Elemento': pos2['checkpointGroupName']})
                    cochemechanical_inspecciones.append({'nºInspecciones(totales)': len(pos2["inspectionCheckPoints"])})
                    cochemechanical_orden_inspecciones.append({'Orden Inspección': inspecciones})
                    cochemechanical_informe.append({'Informe de inspección':pos3['checkpointName']})

                    for i4,pos4 in enumerate(pos3['inspectionCheckPointDetails']):
                        if pos4['isChecked']==True:
                            if pos4['checkpointDetailName']=='Yes':
                                cochemechanical_detalle.append({'Detalle inspección':pos4['checkpointDetailName']+", "+pos3['checkpointName']})
                            else:
                                cochemechanical_detalle.append({'Detalle inspección':pos4['checkpointDetailName']})

                            cochemechanical_imagen.append({'nºImagenes':len(pos3['documents'])})
                            if len(pos3['documents']) >=1:
                                
                                cochemechanical_fileimagen.append({'file_imagen':pos3['documents'][0]['documentPath']})

                            else:
                                cochemechanical_fileimagen.append({'file_imagen':'No_hay_Imagen'})
                                
                                
elementos=pd.DataFrame(cochemechanical_elemento)
inspecciones=pd.DataFrame(cochemechanical_inspecciones)
Orden_inspeccion=pd.DataFrame(cochemechanical_orden_inspecciones)
informe=pd.DataFrame(cochemechanical_informe)
detalle=pd.DataFrame(cochemechanical_detalle)
imagen=pd.DataFrame(cochemechanical_imagen)
fileimagen=pd.DataFrame(cochemechanical_fileimagen)

dfMech=pd.concat([elementos,inspecciones,Orden_inspeccion,informe,detalle,imagen,fileimagen], axis=1)
dfMech

,Elemento,nºInspecciones(totales),Orden Inspección,Informe de inspección,Detalle inspección,nºImagenes,file_imagen
0,Tyres,1,1,Spare Tire,Var,1,https://prvtrprdaopsvava.blob.core.windows.net...


In [134]:
df3['Mech_summary']=[dfMech] 
df3

,marca,modelo,detalles,Fecha,actualMileage,Price,combustible,almacen,ID,imgcar,date,condittion_summary,Mech_summary
0,Suzuki,Vitara,1.4 Boosterjet 4x2 Gl Plus Tek Renk 140HP,2022-02-02T00:00:00,41539,388000.0,Benzin,Basaksehir Warehouse,43922,https://prvtrprdaopsvava.blob.core.windows.net...,2022-02-02,Elemento Subelemento nºInsp...,Elemento nºInspecciones(totales) Orden Ins...


In [135]:
df3['Mech_summary'][0]

,Elemento,nºInspecciones(totales),Orden Inspección,Informe de inspección,Detalle inspección,nºImagenes,file_imagen
0,Tyres,1,1,Spare Tire,Var,1,https://prvtrprdaopsvava.blob.core.windows.net...


## Construyendo dataframe descripcion1, se pasara su analisis a la info general del df, si tiene imagenes,estará contenido en una columna del df_general 

In [136]:
from PIL import Image, ImageDraw
import unicodedata
import glob,os
import re
lista=[]
listai=[]
cochefeature1_feature=[]
cochefeature1_description=[]


cochefeature1_imagen=[]
cochefeature1_fileimagen=[]
#inspeccionar esas inspecciones de cada elemento
for i,pos in enumerate(response_info2['descriptionSummary'].values()):

            for i2,pos2 in enumerate(pos[0]['inspectionCheckPoints']): #son 38 diccionarios
                #print()
                #if len(pos2['inspectionCheckPointDetails'])!=0:               
                #   print("Estamos en el elemento "+str(i2)+" llamado "+ pos2['checkpointName'] + " q tiene "+str(len(pos2['inspectionCheckPointDetails']))+" subelementos")#cada diccionario tiene una lista de diccionarios, componentes del elemento en cuestion
        #ejecutar esto de arriba para ver toooodos los elementos del cohce incluso los vacios de detalles
                
                inspecciones=0
                for i3,pos3 in enumerate(pos2['inspectionCheckPointDetails']):
                    inspecciones=inspecciones+1
                    
                    
                    if pos3['isChecked']==True:
                        #Yakıt Tipi-->fuel type
                        cochefeature1_feature.append({'Feature':pos2["checkpointName"]})
                        cochefeature1_description.append({'Descrption':pos3['checkpointDetailName']})
                        #print(pos3)
                        cochefeature1_imagen.append({'NºImagenes':len(pos2['documents'])})
                        if len(pos2['documents']) >=1:#esto podria ponerse fuera del for pero como solo entra en detalles true lo dejo ahi solo lo chequea 1 vez
                            #print(pos2['documents'][0]['documentPath'])
                            cochefeature1_fileimagen.append({'file_imagen':pos2['documents'][0]['documentPath']})

                        else:
                            cochefeature1_fileimagen.append({'file_imagen':'No_hay_Imagen'})
                    
feature=pd.DataFrame(cochefeature1_feature)
description=pd.DataFrame(cochefeature1_description)

imagen=pd.DataFrame(cochefeature1_imagen)
fileimagen=pd.DataFrame(cochefeature1_fileimagen)

dffeature1=pd.concat([feature,description,imagen,fileimagen], axis=1)
dffeature1
                                


,Feature,Descrption,NºImagenes,file_imagen
0,Body type,SUV,0,No_hay_Imagen
1,Color,Koyu gri,0,No_hay_Imagen
2,Number of doors,5,0,No_hay_Imagen
3,Yakıt Tipi,Benzin,0,No_hay_Imagen
4,Wheel drive,Front Wheel Drive,0,No_hay_Imagen
5,Transmission,Automatic,0,No_hay_Imagen
6,Seating capacity,5,0,No_hay_Imagen


In [137]:
df3['features1_summary']=[dffeature1] 
df3

,marca,modelo,detalles,Fecha,actualMileage,Price,combustible,almacen,ID,imgcar,date,condittion_summary,Mech_summary,features1_summary
0,Suzuki,Vitara,1.4 Boosterjet 4x2 Gl Plus Tek Renk 140HP,2022-02-02T00:00:00,41539,388000.0,Benzin,Basaksehir Warehouse,43922,https://prvtrprdaopsvava.blob.core.windows.net...,2022-02-02,Elemento Subelemento nºInsp...,Elemento nºInspecciones(totales) Orden Ins...,Feature Descrption NºImag...


In [159]:
df3['features1_summary'][0]
#eliminar si no hay imagenes

,Feature,Descrption,NºImagenes,file_imagen
0,Body type,SUV,0,No_hay_Imagen
1,Color,Koyu gri,0,No_hay_Imagen
2,Number of doors,5,0,No_hay_Imagen
3,Yakıt Tipi,Benzin,0,No_hay_Imagen
4,Wheel drive,Front Wheel Drive,0,No_hay_Imagen
5,Transmission,Automatic,0,No_hay_Imagen
6,Seating capacity,5,0,No_hay_Imagen


In [148]:
#convertir a df 
dfeatures1 = pd.DataFrame(columns = df3['features1_summary'][0]['Feature'].values.tolist())
dfeatures1.loc[len(dfeatures1)] = df3['features1_summary'][0]['Descrption'].values.tolist()
#dfeatures1

,Body type,Color,Number of doors,Yakıt Tipi,Wheel drive,Transmission,Seating capacity
0,SUV,Koyu gri,5,Benzin,Front Wheel Drive,Automatic,5


In [160]:
#y añadirla al df
df3=pd.concat([df3,dfeatures1], axis=1)

v=df3['features1_summary'][0]['NºImagenes'].unique()==0
if v[0]==True:
    del df3['features1_summary']
df3

,marca,modelo,detalles,Fecha,actualMileage,Price,combustible,almacen,ID,imgcar,date,condittion_summary,Mech_summary,Body type,Color,Number of doors,Yakıt Tipi,Wheel drive,Transmission,Seating capacity
0,Suzuki,Vitara,1.4 Boosterjet 4x2 Gl Plus Tek Renk 140HP,2022-02-02T00:00:00,41539,388000.0,Benzin,Basaksehir Warehouse,43922,https://prvtrprdaopsvava.blob.core.windows.net...,2022-02-02,Elemento Subelemento nºInsp...,Elemento nºInspecciones(totales) Orden Ins...,SUV,Koyu gri,5,Benzin,Front Wheel Drive,Automatic,5


## Construyendo dataframe features2, estará contenido en una columna del df_general

In [170]:
from PIL import Image, ImageDraw
import unicodedata
import glob,os
import re
lista=[]
listai=[]

cochefeature2_feature=[]
cochefeature2_inspecciones=[]
cochefeature2_orden_inspecciones=[]
cochefeature2_componentfeature2=[]
cochefeature2_detalle=[]
cochefeature2_imagen=[]
cochefeature2_fileimagen=[]
#inspeccionar esas inspecciones de cada elemento
for i,pos in enumerate(response_info2['descriptionSummary'].values()):

            for i2,pos2 in enumerate(pos[0]['inspectionCheckPointsGroups']): #son 38 diccionarios
                #print(pos2['checkpointGroupName'])
                inspecciones=0

                for i3,pos3 in enumerate(pos2['inspectionCheckPoints']):
                    #print(pos3)
                    inspecciones=inspecciones+1
                    cochefeature2_feature.append({'Feature':pos2['checkpointGroupName']})
                    cochefeature2_inspecciones.append({'NºInspecciones':len(pos2['inspectionCheckPoints'])})
                    cochefeature2_orden_inspecciones.append({'Orden Inspección': inspecciones})
                    for i4,pos4 in enumerate(pos3['inspectionCheckPointDetails']):
                    
                        if pos4['isChecked']==True:
                            cochefeature2_componentfeature2.append({'Feature Component':pos3["checkpointName"]})
                            cochefeature2_detalle.append({'Detalle':pos4['checkpointDetailName']})
                            
                            cochefeature2_imagen.append({'NºImagen':len(pos3['documents'])})
                            if len(pos3['documents']) >=1:#esto podria ponerse fuera del for pero como solo entra en detalles true lo dejo ahi solo lo chequea 1 vez
                                cochefeature2_fileimagen.append({'file_imagen':pos3['documents'][0]['documentPath']})
                            else:
                                cochefeature2_fileimagen.append({'file_imagen':'No_hay_Imagen'})
                            
                            
                            
feature=pd.DataFrame(cochefeature2_feature)
inspecciones=pd.DataFrame(cochefeature2_inspecciones)
Orden_inspeccion=pd.DataFrame(cochefeature2_orden_inspecciones)
componentfeature2=pd.DataFrame(cochefeature2_componentfeature2)
cochefeature2_detalle=pd.DataFrame(cochefeature2_detalle)
imagen=pd.DataFrame(cochefeature2_imagen)
fileimagen=pd.DataFrame(cochefeature2_fileimagen)

dfeatures2=pd.concat([feature,inspecciones,Orden_inspeccion,componentfeature2,cochefeature2_detalle,imagen,fileimagen], axis=1)
dfeatures2
                    

,Feature,NºInspecciones,Orden Inspección,Feature Component,Detalle,NºImagen,file_imagen
0,Original Number plates,2,1,Front,Genuine,0,No_hay_Imagen
1,Original Number plates,2,2,Rear,Genuine,0,No_hay_Imagen
2,Colour,1,1,Paint type,Opak,0,No_hay_Imagen


In [171]:
df3['features2_summary']=[dfeatures2] 
#df3['Paint']=[dfeatures2] 
df3

,marca,modelo,detalles,Fecha,actualMileage,Price,combustible,almacen,ID,imgcar,...,condittion_summary,Mech_summary,Body type,Color,Number of doors,Yakıt Tipi,Wheel drive,Transmission,Seating capacity,features2_summary
0,Suzuki,Vitara,1.4 Boosterjet 4x2 Gl Plus Tek Renk 140HP,2022-02-02T00:00:00,41539,388000.0,Benzin,Basaksehir Warehouse,43922,https://prvtrprdaopsvava.blob.core.windows.net...,...,Elemento Subelemento nºInsp...,Elemento nºInspecciones(totales) Orden Ins...,SUV,Koyu gri,5,Benzin,Front Wheel Drive,Automatic,5,Feature NºInspecciones Ord...


In [172]:
df3['features2_summary'][0]

,Feature,NºInspecciones,Orden Inspección,Feature Component,Detalle,NºImagen,file_imagen
0,Original Number plates,2,1,Front,Genuine,0,No_hay_Imagen
1,Original Number plates,2,2,Rear,Genuine,0,No_hay_Imagen
2,Colour,1,1,Paint type,Opak,0,No_hay_Imagen


In [178]:
df3[['marca', 'modelo', 'detalles', 'Fecha', 'actualMileage', 'Price',
       'combustible', 'almacen', 'ID', 'imgcar', 'date',
        'Body type', 'Color', 'Number of doors', 'Yakıt Tipi',
       'Wheel drive', 'Transmission', 'Seating capacity']]

,marca,modelo,detalles,Fecha,actualMileage,Price,combustible,almacen,ID,imgcar,date,Body type,Color,Number of doors,Yakıt Tipi,Wheel drive,Transmission,Seating capacity
0,Suzuki,Vitara,1.4 Boosterjet 4x2 Gl Plus Tek Renk 140HP,2022-02-02T00:00:00,41539,388000.0,Benzin,Basaksehir Warehouse,43922,https://prvtrprdaopsvava.blob.core.windows.net...,2022-02-02,SUV,Koyu gri,5,Benzin,Front Wheel Drive,Automatic,5


In [179]:
df3_1=df3[['marca', 'modelo', 'detalles', 'Fecha', 'actualMileage', 'Price',
       'combustible', 'almacen', 'ID', 'imgcar', 'date',
        'Body type', 'Color', 'Number of doors', 'Yakıt Tipi',
       'Wheel drive', 'Transmission', 'Seating capacity']]

In [39]:
# Converting links to html tags
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

In [180]:
# Rendering the dataframe as HTML table
from IPython.core.display import HTML
HTML(df3_1.to_html(escape=False,formatters=dict(imgcar=path_to_image_html)))

,marca,modelo,detalles,Fecha,actualMileage,Price,combustible,almacen,ID,imgcar,date,Body type,Color,Number of doors,Yakıt Tipi,Wheel drive,Transmission,Seating capacity
0,Suzuki,Vitara,1.4 Boosterjet 4x2 Gl Plus Tek Renk 140HP,2022-02-02T00:00:00,41539,388000.0,Benzin,Basaksehir Warehouse,43922,,2022-02-02,SUV,Koyu gri,5,Benzin,Front Wheel Drive,Automatic,5


In [183]:
df=HTML(df3.to_html(escape=False,formatters=dict(imgcar=path_to_image_html)))

In [184]:
df3_1.to_html('webpage2.html',escape=False, formatters=dict(imgcar=path_to_image_html))